In [1]:
import networkx as nx
import gensim.models
from os import walk
import numpy as np
import pandas as pd
import pickle
import sys
import re
import semanticanalysis as sa
import functools
import itertools
from multiprocessing import Pool
import sys
import pickle

## Single Network

In [35]:
L = 5
G = nx.read_gexf('results/srcnets/trump-99.gexf')
nodes = list(sorted(G.nodes()))
print('\\begin{tabular}{ l |', ' '.join(['c',]*len(G.nodes())), '}')
print('\\hline')
print('sources &', ' & '.join([v.split('_')[0] for v in nodes])+'}', '\\\\')
print('\\hline')
for u in nodes:
    srcname = u.split('_')[0]
    weights = [(v, G.edge[u][v]['weight']) for v in nodes]
    wstrings = ['{0:.3f}'.format(w) if v < u else '-' for v,w in weights]
    print(srcname, '&', ' & '.join(wstrings), '\\\\')
    
print('\\end{tabular}')

\begin{tabular}{ l | c c c c c c c c c c }
\hline
sources & breitbart & cbsnews & cnn & foxnews & huffpo & nytimes & reuters & usatoday & wapo & watimes} \\
\hline
breitbart & - & - & - & - & - & - & - & - & - & - \\
cbsnews & 7.550 & - & - & - & - & - & - & - & - & - \\
cnn & 8.657 & 5.246 & - & - & - & - & - & - & - & - \\
foxnews & 9.473 & 6.592 & 9.247 & - & - & - & - & - & - & - \\
huffpo & 4.477 & 2.373 & 6.016 & 4.827 & - & - & - & - & - & - \\
nytimes & 5.150 & 2.688 & 6.909 & 5.590 & 9.364 & - & - & - & - & - \\
reuters & 4.210 & 2.273 & 5.818 & 4.583 & 9.206 & 9.108 & - & - & - & - \\
usatoday & 6.769 & 9.776 & 4.595 & 5.800 & 2.027 & 2.342 & 1.968 & - & - & - \\
wapo & 4.264 & 2.213 & 5.895 & 4.591 & 9.460 & 9.396 & 9.254 & 1.914 & - & - \\
watimes & 6.432 & 3.331 & 8.325 & 6.983 & 8.319 & 9.027 & 8.076 & 2.923 & 8.238 & - \\
\end{tabular}


In [4]:
G = nx.read_gexf('results/srcnets/trump-500.gexf')
df = pd.DataFrame(nx.to_numpy_matrix(G), index=G.nodes(), columns=G.nodes())
df

,nytimes_pars,cbsnews_pars,usatoday_pars,reuters_pars,cnn_pars,wapo_pars,foxnews_pars,watimes_pars,breitbart_pars,huffpo_pars
nytimes_pars,1.000000,0.421876,0.391960,0.577537,0.658361,0.746602,0.617013,0.708238,0.519877,0.680247
cbsnews_pars,0.421876,1.000000,0.571212,0.368547,0.497727,0.405371,0.618158,0.434685,0.456361,0.364007
usatoday_pars,0.391960,0.571212,1.000000,0.335006,0.379585,0.369974,0.447083,0.415852,0.408051,0.363398
reuters_pars,0.577537,0.368547,0.335006,1.000000,0.535347,0.583582,0.485657,0.554962,0.430513,0.533954
cnn_pars,0.658361,0.497727,0.379585,0.535347,1.000000,0.673022,0.742796,0.658371,0.627762,0.577464
wapo_pars,0.746602,0.405371,0.369974,0.583582,0.673022,1.000000,0.602966,0.748504,0.507554,0.693967
foxnews_pars,0.617013,0.618158,0.447083,0.485657,0.742796,0.602966,1.000000,0.648305,0.626897,0.539643
watimes_pars,0.708238,0.434685,0.415852,0.554962,0.658371,0.748504,0.648305,1.000000,0.527085,0.653886
breitbart_pars,0.519877,0.456361,0.408051,0.430513,0.627762,0.507554,0.626897,0.527085,1.000000,0.439645
huffpo_pars,0.680247,0.364007,0.363398,0.533954,0.577464,0.693967,0.539643,0.653886,0.439645,1.000000


## Network Comparison

In [5]:
L = 5
H = nx.read_gexf('results/srcnets/trump-0.gexf')
G = nx.read_gexf('results/srcnets/trump-999.gexf')
nodes = list(sorted(G.nodes()))
print('\\begin{tabular}{ l |', ' '.join(['c',]*len(G.nodes())), '}')
print('\\hline')
print('sources &', ' & '.join([v.split('_')[0] for v in nodes])+'}', '\\\\')
print('\\hline')
for u in nodes:
    srcname = u.split('_')[0]
    weights = [(v, G.edge[u][v]['weight']-H.edge[u][v]['weight']) for v in nodes]
    wstrings = ['{0:.3f}'.format(w) if v < u else '-' for v,w in weights]
    print(srcname, '&', ' & '.join(wstrings), '\\\\')
    
print('\\end{tabular}')

\begin{tabular}{ l | c c c c c c c c c c }
\hline
sources & breitbart & cbsnews & cnn & foxnews & huffpo & nytimes & reuters & usatoday & wapo & watimes} \\
\hline
breitbart & - & - & - & - & - & - & - & - & - & - \\
cbsnews & 0.042 & - & - & - & - & - & - & - & - & - \\
cnn & -0.047 & 0.102 & - & - & - & - & - & - & - & - \\
foxnews & -0.054 & 0.061 & -0.024 & - & - & - & - & - & - & - \\
huffpo & -0.107 & 0.115 & -0.068 & -0.039 & - & - & - & - & - & - \\
nytimes & -0.045 & 0.148 & -0.032 & -0.008 & -0.110 & - & - & - & - & - \\
reuters & 0.016 & 0.155 & -0.013 & -0.016 & -0.076 & -0.036 & - & - & - & - \\
usatoday & 0.091 & -0.034 & 0.048 & 0.051 & 0.119 & 0.114 & 0.156 & - & - & - \\
wapo & -0.062 & 0.126 & -0.027 & -0.042 & -0.121 & -0.085 & -0.052 & 0.121 & - & - \\
watimes & -0.078 & 0.099 & -0.085 & -0.073 & -0.079 & -0.072 & -0.031 & 0.134 & -0.061 & - \\
\end{tabular}
